In [36]:
import os.path

import torch
import pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [37]:
train_df = pd.read_csv(r'data/WASSA-2017/train/anger-ratings-0to1.train.txt', sep='\t', header=0)
test_df = pd.read_csv(r'data/WASSA-2017/test/anger-ratings-0to1.test.target.txt', sep='\t', header=0)

In [38]:
train_df.head()

,id,tweet,tweettype,score
0,10000,How the fu*k! Who the heck! moved my fridge!.....,anger,0.938
1,10001,So my Indian Uber driver just called someone t...,anger,0.896
2,10002,@DPD_UK I asked for my parcel to be delivered ...,anger,0.896
3,10003,so ef whichever butt wipe pulled the fire alar...,anger,0.896
4,10004,Don't join @BTCare they put the phone down on ...,anger,0.896


 ## Load Data

In [39]:
import glob
from tqdm import tqdm
import os

TRAIN_DIR = 'data/WASSA-2017/train'
TEST_DIR = 'data/WASSA-2017/test'

FILE_PATTERN = '*.txt'
LABEL_SEPERATOR = '-'

def get_label_from_filename(filename, seperator=LABEL_SEPERATOR):
    base_name = os.path.basename(filename)
    label = base_name.split(seperator)[0]
    return label.lower()

def load_data(data_dir, pattern, seperator):
    all_files = glob.glob(os.path.join(data_dir, pattern))
    if not all_files:
        raise FileNotFoundError(f"No files found matching '{pattern}' in directory {data_dir}")

    df_list = []
    print(f"loading files from {data_dir}")
    for filepath in tqdm(all_files, desc="Reading files"):
        try:
            temp_df = pd.read_csv(filepath, sep='\t', header=0)
            label = get_label_from_filename(filepath, seperator)
            temp_df['emotion'] = label
            df_list.append(temp_df[['tweet', 'emotion']])

        except Exception as e:
            print(f"Error processing file {filepath}: {e}")
            continue

    if not df_list:
        raise ValueError(f"No dataframes were created from files in {data_dir}")

    combined_df = pd.concat(df_list, ignore_index=True)
    print(f"Loaded and combined {len(combined_df)} samples")
    print(f"Found emotions: {combined_df['emotion'].unique().tolist()}")
    return combined_df

In [40]:
train_df = load_data(data_dir=TRAIN_DIR, pattern=FILE_PATTERN, seperator=LABEL_SEPERATOR)
test_df = load_data(data_dir=TEST_DIR, pattern=FILE_PATTERN, seperator=LABEL_SEPERATOR)

loading files from data/WASSA-2017/train


Reading files: 100%|██████████| 4/4 [00:00<00:00, 167.02it/s]


Loaded and combined 3613 samples
Found emotions: ['anger', 'fear', 'joy', 'sadness']
loading files from data/WASSA-2017/test


Reading files: 100%|██████████| 4/4 [00:00<00:00, 154.60it/s]

Loaded and combined 3142 samples
Found emotions: ['anger', 'fear', 'joy', 'sadness']


In [41]:
train_df

,tweet,emotion
0,How the fu*k! Who the heck! moved my fridge!.....,anger
1,So my Indian Uber driver just called someone t...,anger
2,@DPD_UK I asked for my parcel to be delivered ...,anger
3,so ef whichever butt wipe pulled the fire alar...,anger
4,Don't join @BTCare they put the phone down on ...,anger
...,...,...
3608,@VivienLloyd Thank you so much! Just home - st...,sadness
3609,Just put the winter duvet on ☃️❄️🌬☔️,sadness
3610,@SilkInSide @TommyJoeRatliff that's so pretty!...,sadness
3611,@BluesfestByron second artist announcement loo...,sadness


## Preprocess Data

In [42]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.preprocessing import LabelEncoder
import string
import numpy as np
from collections import Counter

In [43]:
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    print("Downloading NLTK 'punkt' tokenizer...")
    nltk.download('punkt_tab')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    print("Downloading NLTK 'stopwords'...")
    nltk.download('stopwords')


In [44]:
USE_STEMMING = True
MIN_WORD_FREQ = 1

# --- Special Tokens ---
PAD_TOKEN = "<PAD>"
UNK_TOKEN = "<UNK>"

In [45]:
def preprocess_text(text, processor, stop_words_set):
    text = text.lower()
    tokens = word_tokenize(text)
    # remove punctuations and stop words
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words_set]

    if processor:
         if isinstance(processor, SnowballStemmer):
             # Reduce words to its stem word
             processed_tokens = [processor.stem(word) for word in tokens]
    else:
        processed_tokens = tokens

    return processed_tokens

def build_vocab(processed_docs, min_freq=1):
    """Maps words to indices"""
    word_counts = Counter(token for doc in processed_docs for token in doc)
    vocab = [word for word, count in word_counts.items() if count >= min_freq]
    word_to_idx = {PAD_TOKEN: 0, UNK_TOKEN: 1}
    for i, word in enumerate(vocab): word_to_idx[word] = i + 2
    idx_to_word = {idx: word for word, idx in word_to_idx.items()}
    vocab_size = len(word_to_idx)
    print(f"Total vocabulary size (including {PAD_TOKEN}, {UNK_TOKEN}): {vocab_size}")

    return word_to_idx, idx_to_word, vocab_size

def convert_docs_to_ids(processed_docs, word_to_idx):
    """Converts list of processed tokens into lists of integer IDs"""
    docs_as_ids = []
    for doc in tqdm(processed_docs, desc="Converting docs to IDs"):
        docs_as_ids.append([word_to_idx.get(token, word_to_idx[UNK_TOKEN]) for token in doc])

    return docs_as_ids

In [46]:
raw_train, labels_train_str = train_df['tweet'].tolist(), train_df['emotion'].tolist()
raw_test , labels_test_str = test_df['tweet'].tolist(), test_df['emotion'].tolist()

In [47]:
stop_words_set = set(stopwords.words('english'))
stop_words_set.update(string.punctuation)
processor = SnowballStemmer('english') if USE_STEMMING else None

In [48]:
print("\nProcessing training documents...")
processed_train = [preprocess_text(doc, processor, stop_words_set) for doc in tqdm(raw_train)]
print("\nProcessing test documents...")
processed_test = [preprocess_text(doc, processor, stop_words_set) for doc in tqdm(raw_test)]


Processing training documents...


100%|██████████| 3613/3613 [00:01<00:00, 3265.53it/s]



Processing test documents...


100%|██████████| 3142/3142 [00:00<00:00, 3483.17it/s]


In [49]:
print("\nBuilding vocabulary...")
all_processed_docs = processed_train + processed_test
word_to_idx, idx_to_word, vocab_size = build_vocab(all_processed_docs, MIN_WORD_FREQ)


Building vocabulary...
Total vocabulary size (including <PAD>, <UNK>): 10919


In [50]:
print("\nConverting documents to integer sequences...")
train_ids = convert_docs_to_ids(processed_train, word_to_idx)
test_ids = convert_docs_to_ids(processed_test, word_to_idx)


Converting documents to integer sequences...


Converting docs to IDs: 100%|██████████| 3142/3142 [00:00<00:00, 233190.06it/s]


In [51]:
print("\nProcessing target labels...")
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(labels_train_str)
y_test = label_encoder.transform(labels_test_str)


Processing target labels...


In [52]:
num_classes = len(label_encoder.classes_)
print(f"Number of classes: {num_classes}")
print(f"Class mapping (string -> integer): {dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))}")

Number of classes: 4
Class mapping (string -> integer): {np.str_('anger'): np.int64(0), np.str_('fear'): np.int64(1), np.str_('joy'): np.int64(2), np.str_('sadness'): np.int64(3)}


In [53]:
print("\nPreprocessing complete!")
print(f"Number of training sequences: {len(train_ids)}")
print(f"Number of test sequences: {len(test_ids)}")
print(f"Vocabulary Size: {vocab_size}")
print(f"Number of classes: {num_classes}")


Preprocessing complete!
Number of training sequences: 3613
Number of test sequences: 3142
Vocabulary Size: 10919
Number of classes: 4


## Dataset and DataLoader

In [55]:
from torch.utils.data import Dataset, DataLoader

class TweetDataset(Dataset):
    def __init__(self, sequences, labels):
        """
        :param sequences: Preprocessed sequences (samples)
        :param labels: Corresponding one hot encoded label for each sequence
        """

        if len(sequences) != len(labels):
            raise ValueError("Sequences and labels must have the same length")

        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        """
        :param idx: index of sample to be found
        :return: sample and its corresponding label as tensors
        """
        sequence = self.sequences[idx]
        label = self.labels[idx]

        sequence_tensor = torch.tensor(sequence, dtype=torch.long)
        label_tensor = torch.tensor(label, dtype=torch.long)

        return sequence_tensor, label_tensor

In [56]:
train_set = TweetDataset(train_ids, y_train)
test_set = TweetDataset(test_ids, y_test)

In [57]:
len(train_set)

3613

In [77]:
train_set[0][1]

tensor(0)

In [64]:
from torch.nn.utils.rnn import pad_sequence

PAD_INDEX = 0

def collate_batch(batch):
    """
    Collates data samples into batches.

    Args:
        batch (list of tuple): A list where each element is a tuple
                               (sequence_tensor, label_tensor) from the Dataset.
    Returns:
        tuple: A tuple containing:
               - sequences_padded (Tensor): Batch of padded sequences
                 (batch_size, max_len).
               - labels (Tensor): Batch of labels (batch_size,).
    """
    label_list, sequence_list = [], []
    for (_sequence, _label) in batch:
        label_list.append(_label)
        sequence_list.append(_sequence) # sequence is already a tensor here

    # Pad sequences
    # batch_first=True makes output (batch_size, max_seq_len)
    sequences_padded = pad_sequence(sequence_list, batch_first=True, padding_value=PAD_INDEX)

    # Stack labels
    labels = torch.stack(label_list)

    return sequences_padded, labels

In [65]:
PAD_INDEX = word_to_idx.get(PAD_TOKEN, 0)
BATCH_SIZE = 64

train_loader = DataLoader(
    train_set,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_batch,
)

test_loader = DataLoader(
    test_set,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_batch
)

In [71]:
X, y = next(iter(train_loader))
print(X.shape)
print(y.shape)

torch.Size([64, 16])
torch.Size([64])


## Model

In [83]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embed_len, hidden_dim, num_layers, num_classes):
        super().__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_len)
        self.rnn = nn.RNN(input_size=embed_len, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        embeddings = self.embedding_layer(x)
        output, hidden = self.rnn(embeddings)
        return self.linear(output[:, -1])

In [81]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    """Performs one training epoch."""
    model.train()  # Set model to training mode (enables dropout, etc.)
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    # Wrap dataloader with tqdm for a progress bar
    progress_bar = tqdm(dataloader, desc="Training", leave=False)

    for sequences_batch, labels_batch in progress_bar:
        # Move data to the selected device
        sequences_batch = sequences_batch.to(device)
        labels_batch = labels_batch.to(device)

        # --- Forward Pass ---
        logits = model(sequences_batch)

        # --- Calculate Loss ---
        loss = criterion(logits, labels_batch)

        # --- Backward Pass and Optimization ---
        optimizer.zero_grad() # Clear previous gradients
        loss.backward()       # Compute gradients
        optimizer.step()      # Update model weights

        # --- Track Metrics ---
        running_loss += loss.item() * sequences_batch.size(0) # Accumulate weighted loss
        total_samples += labels_batch.size(0)

        # Calculate accuracy for the batch
        with torch.no_grad(): # No need to track gradients for accuracy calculation
            preds = torch.argmax(logits, dim=1)
            correct_predictions += torch.sum(preds == labels_batch).item()

        # Update progress bar postfix
        progress_bar.set_postfix(loss=loss.item(), acc=correct_predictions/total_samples)


    epoch_loss = running_loss / total_samples
    epoch_acc = correct_predictions / total_samples
    return epoch_loss, epoch_acc

# --- Evaluation Function ---
def evaluate(model, dataloader, criterion, device):
    """Evaluates the model on the given dataset."""
    model.eval()  # Set model to evaluation mode (disables dropout, etc.)
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    # Wrap dataloader with tqdm for a progress bar
    progress_bar = tqdm(dataloader, desc="Evaluating", leave=False)

    with torch.no_grad(): # Disable gradient calculation during evaluation
        for sequences_batch, labels_batch in progress_bar:
            # Move data to the selected device
            sequences_batch = sequences_batch.to(device)
            labels_batch = labels_batch.to(device)

            # --- Forward Pass ---
            logits = model(sequences_batch)

            # --- Calculate Loss ---
            loss = criterion(logits, labels_batch)

            # --- Track Metrics ---
            running_loss += loss.item() * sequences_batch.size(0)
            total_samples += labels_batch.size(0)

            # Calculate accuracy for the batch
            preds = torch.argmax(logits, dim=1)
            correct_predictions += torch.sum(preds == labels_batch).item()

            # Update progress bar postfix
            progress_bar.set_postfix(loss=loss.item(), acc=correct_predictions/total_samples)


    epoch_loss = running_loss / total_samples
    epoch_acc = correct_predictions / total_samples
    return epoch_loss, epoch_acc

In [89]:
LR = 0.001
EPOCHS = 50
embed_len = 128
hidden_size = 50
num_layers = 3

model = RNN(vocab_size, embed_len, hidden_size, num_layers, num_classes).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()

# Training loop
for epoch in range(EPOCHS):
    print(f"\n--- Epoch {epoch+1}/{EPOCHS} ---")

    train_loss, train_acc = train_one_epoch(model, train_loader, loss_fn, optimizer, device)
    print(f"Epoch {epoch+1} Training   -> Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")

    val_loss, val_acc = evaluate(model, test_loader, loss_fn, device) # Using test_loader as validation
    print(f"Epoch {epoch+1} Validation -> Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}")

    # Optional: Add logic here to save the best model based on validation accuracy/loss
    # e.g., if val_acc > best_val_acc: save model state_dict

print("\n--- Training Complete ---")


--- Epoch 1/50 ---


Epoch 1 Training   -> Loss: 1.3814, Accuracy: 0.3105


Epoch 1 Validation -> Loss: 1.3764, Accuracy: 0.3148

--- Epoch 2/50 ---


Epoch 2 Training   -> Loss: 1.3753, Accuracy: 0.3144


Epoch 2 Validation -> Loss: 1.3765, Accuracy: 0.3154

--- Epoch 3/50 ---


...

Epoch 48 Validation -> Loss: 1.9777, Accuracy: 0.4892

--- Epoch 49/50 ---


Epoch 49 Training   -> Loss: 0.3533, Accuracy: 0.8993


Epoch 49 Validation -> Loss: 1.8649, Accuracy: 0.5102

--- Epoch 50/50 ---


Epoch 50 Training   -> Loss: 0.3051, Accuracy: 0.9134


Epoch 50 Validation -> Loss: 1.9191, Accuracy: 0.5258

--- Training Complete ---
